In [1]:
# imports
from typing import Any

import requests
import json

import open3d as o3d
import numpy as np

import subprocess

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
# get the id of the first annotation
url = "http://localhost:4321/api/annotations/selected/points"
response = requests.get(url)
id = json.loads(response.text)[0]
print(id)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# --- helper functions ---
def applyTrafoToSurface(port: int, surface_id: str, transformation_matrix: np.ndarray) -> None:
    """
    Applies a transformation matrix to the surface with the given ID using an HTTP PUT request.
    
    Args:
        port (int): The port number of the API server.
        surface_id (str): The unique identifier of the surface.
        transformation_matrix (np.ndarray): A 2D NumPy array representing the transformation matrix.
    """
    url = f"http://localhost:{port}/api/surfaces/{surface_id}/transformation"

    payload: dict[str, Any] = {
        "forward": transformation_matrix.tolist()
    }

    headers: dict[str, str]= {
        "Content-Type": "application/json"
    }

    response = requests.put(url, json=payload, headers=headers)
    response.raise_for_status() 
    print(response.text)    

def writePcdObj (outputFile, pcd) -> None:
    """
    Writes a point cloud to a .pcd and .obj file.
    """
    with open(outputFile, "w") as f:
        for point in pcd.points:
            f.write(f"v {point[0]} {point[1]} {point[2]}\n")

def openFileInMeshlab (file) -> None:
    """
    Opens a file in Meshlab.
    """
    # open cutout in MeshLab
    try:
        subprocess.Popen(["C:/Program Files/vcg/MeshLab/meshlab.exe", file])
        print("MeshLab started successfully.")
    except FileNotFoundError:
        print("MeshLab executable not found. Please ensure it is installed and in your system PATH.")


In [ ]:
tgt_points_id = "3d87d707-5735-4d7d-a401-679ae2a748b5"
src_points_id = "4b854809-1601-4e7e-aa78-85aa19fa43d3"

url = "http://localhost:4322/api/annotations/3d87d707-5735-4d7d-a401-679ae2a748b5/points"

response = requests.get(url)
tgt_points = np.array(json.loads(response.text))

print(f"tgt points: {tgt_points}")

url = "http://localhost:4321/api/annotations/4b854809-1601-4e7e-aa78-85aa19fa43d3/points"

response = requests.get(url)
src_points = np.array(json.loads(response.text))

print(f"src points: {src_points}")


In [ ]:
# Convert numpy arrays to Open3D PointClouds
source_pcd = o3d.geometry.PointCloud()
source_pcd.points = o3d.utility.Vector3dVector(src_points)

target_pcd = o3d.geometry.PointCloud()
target_pcd.points = o3d.utility.Vector3dVector(tgt_points)

# Define correspondences as index pairs
correspondences = np.array([[0, 0], [1, 1], [2, 2], [3, 3]])
correspondences_o3d = o3d.utility.Vector2iVector(correspondences)

# Create a TransformationEstimationPointToPoint object
estimation = o3d.pipelines.registration.TransformationEstimationPointToPoint()

# Compute the rigid transformation matrix
transformation_matrix = estimation.compute_transformation(target_pcd, source_pcd, correspondences_o3d)

# Print the transformation matrix
print("Computed Transformation Matrix:")
print(transformation_matrix)

In [ ]:
# Apply transformation to target surface
tgt_surface_id = "191cef90-bcbc-4acb-9318-ae11aa6355d4"
applyTrafoToSurface(4321, tgt_surface_id, transformation_matrix)


In [ ]:
cut_annotation_id = "4c2224dd-96d8-4921-824b-f82f2c1e3812"
url = "http://localhost:4321/api/queries/queryAnnotationAsObj"

# retrieve the cutout geometry
payload = {
    "annotationId": cut_annotation_id, 
    "queryAttributes":[], 
    "distanceToPlane": 100000.0,
    "outputReferenceFrame": "global",
    "outputGeometryType": "mesh"
}
print(payload)
response = requests.post(url, json=payload)
response.raise_for_status() 
filename = "./coutout.obj"
file = open(filename, "w")
file.write(response.text)
file.close()

In [ ]:
# ICP fine registration

# Load the SRC and TGT meshes
src_mesh = o3d.io.read_triangle_mesh(filename)
tgt_mesh = o3d.io.read_triangle_mesh("G:\PRo3D\AI-Mars-3D\Co-Registration\packageGP\m2020-zcam-kodiak\source\model_4b_kodiak_100m_zcam_scam.obj")

# Convert meshes to point clouds for registration
src_pcd = src_mesh.sample_points_uniformly(number_of_points=100000)
tgt_pcd = tgt_mesh.sample_points_uniformly(number_of_points=100000)

# Compute normals for both point clouds
# src_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
# tgt_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Define the pre-alignment (Helmert transformation)
pre_alignment = transformation_matrix

# Apply the pre-alignment transformation to the TGT point cloud
tgt_pcd.transform(pre_alignment)

writePcdObj("src_pcd.obj", src_pcd)
writePcdObj("tgt_pcd.obj", tgt_pcd)

openFileInMeshlab("src_pcd.obj")
openFileInMeshlab("tgt_pcd.obj")

# Perform fine registration using ICP
icp_result = o3d.pipelines.registration.registration_icp(
    tgt_pcd, src_pcd, max_correspondence_distance=0.1,
    init=np.eye(4),
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane()
)

# Print the transformation matrices
print("Pre-Alignment (Helmert) Transformation:")
print(pre_alignment)

print("\nICP Refinement Transformation:")
print(icp_result.transformation)

In [ ]:
# Apply final trafo to target surface
tgt_surface_id = "191cef90-bcbc-4acb-9318-ae11aa6355d4"

url = f"http://localhost:4321/api/surfaces/{tgt_surface_id}/transform"

payload = {
    "forward": icp_result.transformation.tolist()
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)
response.raise_for_status() 
print(response.text)